<a href="https://colab.research.google.com/github/tydesharnais/Genre_Classifacation/blob/test_outputs/Incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.layers import GaussianNoise
from keras.regularizers import l1,l2
from keras.wrappers.scikit_learn import KerasClassifier


import sys
import csv

csv.field_size_limit(sys.maxsize)



from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score, roc_curve 

import matplotlib.pyplot as plt
from sklearn.manifold import Isomap

from sklearn.datasets import load_digits #mnist

from prepare import load_Data, handle_nulls, prepare_df, change_dtypes
from preprocessing import spotify_split, scale_data, create_features

In [ ]:
real_DF = pd.read_csv('final_dataframe_with_mfccs.csv', engine='python', error_bad_lines=False)
real_DF = real_DF.drop_duplicates()
real_DF = create_features(real_DF)
real_DF = change_dtypes(real_DF)
#.loc accepts the same slice notation that Python lists do for both row and columns. Slice notation being start:stop:step
real_DF = real_DF.drop(columns=['song', 'artist', 
                        'year', 'duration_minutes', 
                        'duration_seconds', 'explicit','Unnamed: 0', 'has_feat'])
real_DF = real_DF.reindex(columns = [col for col in real_DF.columns if col != 'genre'] + ['genre'])
real_DF['genre'] = real_DF['genre'].str.split(',').str[0]
real_DF['genre'] = real_DF['genre'].astype('string')
real_DF.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1553 entries, 0 to 1552
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0.1      1553 non-null   int64  
 1   duration_ms       1553 non-null   int64  
 2   popularity        1553 non-null   int64  
 3   danceability      1553 non-null   float64
 4   energy            1553 non-null   float64
 5   key               1553 non-null   int64  
 6   loudness          1553 non-null   float64
 7   mode              1553 non-null   int64  
 8   speechiness       1553 non-null   float64
 9   acousticness      1553 non-null   float64
 10  instrumentalness  1553 non-null   float64
 11  liveness          1553 non-null   float64
 12  valence           1553 non-null   float64
 13  tempo             1553 non-null   float64
 14  rolloff-mean      1553 non-null   float64
 15  rolloff-var       1553 non-null   float64
 16  bandwidth-mean    1553 non-null   float64


In [ ]:
from sklearn.preprocessing import StandardScaler

genre_Classification = real_DF.iloc[:,-1]
convert = LabelEncoder()
y = convert.fit_transform(genre_Classification)
print('Features')
print(real_DF.iloc[:,:-1])
print('Labels')
real_DF.iloc[:,-1] = y
print(real_DF.iloc[:,-1])
set_df_index = real_DF[real_DF.genre == 'set()'].index
real_DF.drop(set_df_index, inplace=True)

fit = StandardScaler()
new_Y = real_DF['genre']
X = real_DF.loc[:, real_DF.columns != 'genre']
cols = X.columns
np_Scaled = fit.fit_transform(X)
X = pd.DataFrame(np_Scaled, columns = cols)

n_classes = new_Y.nunique()
print(n_classes)
print(new_Y)


Features
      duration_ms  popularity  danceability  energy  key  loudness  mode  \
0          167066          79         0.434   0.897    0    -4.918     1   
1          250546          66         0.529   0.496    7    -9.007     1   
2          253733          69         0.706   0.888    2    -6.959     1   
3          284200          86         0.949   0.661    5    -4.244     0   
4          258560          68         0.708   0.772    7    -4.264     1   
...           ...         ...           ...     ...  ...       ...   ...   
1548       181026          79         0.842   0.734    1    -5.065     0   
1549       178426          78         0.552   0.702    9    -5.707     1   
1550       200593          69         0.847   0.678    9    -8.635     1   
1551       171029          75         0.741   0.520    8    -7.513     1   
1552       215280          85         0.695   0.762    0    -3.497     1   

      speechiness  acousticness  instrumentalness  ...  mfcc16-mean  \
0      

In [ ]:
from keras.layers.attention.multi_head_attention import activation
Y = real_DF[['mfcc_Array']].values
Y.shape


(1553, 1)

In [ ]:
final_mfcc= final_mfcc[...,np.newaxis]
final_mfcc[0][0][0][0]
#(1, 517, 20)


array([[[[[[['[[[-29.065750122070312, 110.05075073242188, -26.512069702148438, 36.469940185546875, 4.243959426879883, 4.164793014526367, -1.6815299987792969, 19.391834259033203, -4.167994499206543, 16.56667709350586, -7.30402135848999, 12.830231666564941, 5.836133003234863, -4.02957010269165, -1.1164729595184326, -18.22951889038086, -7.467351913452148, 7.113283157348633, -7.415215492248535, 14.835192680358887], [3.263289451599121, 105.31986999511719, -22.44659423828125, 50.47523498535156, 4.053096771240234, 7.072846412658691, 0.1143338680267334, 14.312956809997559, -9.913056373596191, 12.41868782043457, -2.2498857975006104, 19.91828155517578, 1.6193952560424805, -2.9181084632873535, 2.6528892517089844, -8.78325366973877, -10.509328842163086, 1.5936769247055054, -9.396940231323242, 16.087942123413086], [6.649626731872559, 101.24408721923828, -9.48000431060791, 64.46022033691406, 6.440215110778809, 7.270536422729492, 1.718724012374878, 14.247885704040527, -1.7460825443267822, 15.25941467